In [41]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [42]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-06-16 21:14:58--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  95.0MB/s    in 0.3s    

2024-06-16 21:14:59 (95.0 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [51]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [52]:
# add your code here - consider creating a new cell for each section of code

# Filter users who have given at least 200 ratings
num_users = df_ratings.groupby('user')["rating"].count()
num_users = num_users[num_users >= 200].index
df_user_counts = df_ratings[df_ratings['user'].isin(num_users)]

# Filter books that have received at least 100 ratings
num_ratings = df_ratings.groupby('isbn')["rating"].count()
num_ratings = num_ratings[num_ratings >= 100].index
df_book_counts = df_ratings[df_ratings['isbn'].isin(num_ratings)]

# Merge the filtered data
df_filtered = df_user_counts[df_user_counts['isbn'].isin(df_book_counts['isbn'])]

# Merge with book information
df_book_ratings = pd.merge(df_filtered, df_books, on="isbn", how="inner")

# Resulting DataFrame
print(df_book_ratings.head())







     user        isbn  rating  \
0  277427  002542730X    10.0   
1    3363  002542730X     0.0   
2   11676  002542730X     6.0   
3   12538  002542730X    10.0   
4   13552  002542730X     0.0   

                                               title             author  
0  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  
1  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  
2  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  
3  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  
4  Politically Correct Bedtime Stories: Modern Ta...  James Finn Garner  


In [53]:
df_book_ratings = df_book_ratings.drop_duplicates(['title', 'user'])


In [54]:
df_pivot_book_ratings = df_book_ratings.pivot(index='title', columns='user', values='rating').fillna(0)
df_matrix_book_ratings = csr_matrix(df_pivot_book_ratings.values)

In [55]:
# Creating and training the model
kNN = NearestNeighbors(metric='cosine', algorithm='brute', p=2).fit(df_matrix_book_ratings)


In [61]:
get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', 0.7939835],
  ['The Witching Hour (Lives of the Mayfair Witches)', 0.74486566],
  ['Interview with the Vampire', 0.73450685],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.53763384],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.51784116]]]

In [59]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    recommended_books = [book, []]

    # Get the distances and indices of the nearest neighbors, use 6 because first one will be itself
    dist, idx = kNN.kneighbors([df_pivot_book_ratings.loc[book].values], n_neighbors=6, return_distance=True)

    # Extract the recommended book titles and distances, excluding the first one (the book itself)
    for i in range(len(idx[0])-1, 0, -1):
        recommended_books[1].append([df_pivot_book_ratings.index[idx[0][i]], dist[0][i]])
    return recommended_books

In [60]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
